# <img src=https://i.ibb.co/wJW61Y2/Used-cars.jpg width="700" height="200">

## <p style="background-color:#FDFEFE; font-family:newtimeroman; color:#060108; font-size:200%; text-align:center; border-radius:10px 10px;">The Capstone Project of Data Analytics Module</p>

## <p style="background-color:#FDFEFE; font-family:newtimeroman; color:#060108; font-size:150%; text-align:center; border-radius:10px 10px;">Car Price Prediction EDA</p>

## Introduction
Welcome to "***AutoScout Data Analysis Project***". This is the capstone project of ***Data Analysis*** Module. **Auto Scout** data which using for this project, scraped from the on-line car trading company in 2019, contains many features of 9 different car models. In this project, you will have the opportunity to apply many commonly used algorithms for Data Cleaning and Exploratory Data Analysis by using many Python libraries such as Numpy, Pandas, Matplotlib, Seaborn, Scipy you will analyze clean dataset.

The project consists of 3 parts:
* First part is related with 'data cleaning'. It deals with Incorrect Headers, Incorrect Format, Anomalies, Dropping useless columns.
* Second part is related with 'filling data'. It deals with Missing Values. Categorical to numeric transformation is done.
* Third part is related with 'handling outliers of data' via Visualisation libraries. Some insights are extracted.


### Columns:

**General Columns**
* url: url of autos
* short_description, description: Description of autos (in English and German) written by users

**Categorical Columns**
* make_model, make, model: Model of autos. Ex:Audi A1
* body_type, body: Body type of autos Example: van, sedans
* vat: VAT deductible, price negotiable
* registration, first_registration: First registration date and year of autos.
* prev_owner, previous_owners: Number of previous owners
* type: new or used
* next_inspection, inspection_new: information about inspection (inspection date,..)
* body_color, body_color_original: Color of auto Ex: Black, red
* paint_type: Paint type of auto Ex: Metallic, Uni/basic
* upholstery: Upholstery information (texture, color)
* gearing_type: Type of gear Ex: automatic, manual
* fuel : fuel type Ex: diesel, benzine
* co2_emission, emission_class, emission_label: emission information
* drive_chain: drive chain Ex: front,rear, 4WD
* consumption: consumption of auto in city, country and combination (lt/100 km)
* country_version
* entertainment_media
* safety_security
* comfort_convenience
* extras

**Quantitative Columns**
* price: Price of cars
* km: km of autos
* hp: horsepower of autos (kW)
* displacement: displacement of autos (cc)
* warranty: warranty period (month)
* weight: weight of auto (kg)
* nr_of_doors: number of doors
* nr_of_seats : number of seats
* cylinders: number of cylinders
* gears: number of gears



In [88]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from skimpy import clean_columns

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline

plt.rcParams["figure.figsize"] = (10,6)

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
import re

In [89]:
df=pd.read_json("scout_car.json",lines=True)

In [90]:
df.columns=df.columns.str.strip()

In [91]:
df.isnull().sum()*100/df.shape[0]

url                               0.00
make_model                        0.00
short_description                 0.29
body_type                         0.38
price                             0.00
vat                              28.35
km                                0.00
registration                      0.00
prev_owner                       42.89
kW                              100.00
hp                                0.00
Type                              0.01
Previous Owners                  41.71
Next Inspection                  77.79
Inspection new                   75.30
Warranty                         34.05
Full Service                     48.39
Non-smoking Vehicle              54.92
null                              0.00
Make                              0.00
Model                             0.00
Offer Number                     19.94
First Registration               10.03
Body Color                        3.75
Paint Type                       36.26
Body Color Original      

### 1) Null-URL

*All of data in "Null-column"  are nun-value and each row has own Url. It means when we would transform our data to numeric there will be as columns-number as rows number. That's why there will be dropped.*

In [92]:
df.drop(["null","url"],axis=1,inplace=True)

### 2) Make_Model-Make-Model

In [93]:
(df["make_model"].str.split(" ").str[0]==df["Make"].str.strip()).value_counts(dropna=False)

True    15919
dtype: int64

In [94]:
(df["make_model"].str.split(" ").str[1]==df["Model"].str.strip()).value_counts(dropna=False)

False    15919
dtype: int64

*The values on make_model and the values  on the other two columns are equal.*

**As seen, the "make_model" column gives us a more detailed information for our analysis so after the examination of the following columns, "Make" & "Model", we will decide which one/s we will continue.**

In [95]:
df.drop(["Make","Model"],axis=1,inplace=True)

### 3) Short description 

In [96]:
df['short_description'].value_counts(dropna=False)

SPB 1.6 TDI 116 CV S tronic Sport                 64
NaN                                               46
1.4 66kW (90CV) Selective                         40
MOVE KLIMA CD USB ALLWETTER BLUETOOTH             38
SPB 30 TDI S tronic Business                      35
                                                  ..
K Sports Tourer Business **Navi*Voll**             1
1.0 Turbo ecoFLEX Automatic Elective Park Navi     1
1.6 CDTI Business *Navi*Automatik*Spurh.*          1
1.6 136 CV Automatica Navi E 6 Garanzia Full       1
TCe 225 EDC GPF LIM Deluxe Pano,RFK                1
Name: short_description, Length: 10002, dtype: int64

* The displacement values in short description column was extracted.

In [97]:
df['sd_disp']=df['short_description'].str.findall('\d\.\d').str[0].astype(float)

**We have pulled these values and can use these to fill nan_values on the Displacement column.**

In [98]:
df['sd_disp'] = df['sd_disp']*1000

In [99]:
df.loc[df['sd_disp']<800,'sd_disp'] = np.nan # It is not normal there are motor under this quantity

### 4) Displacement

In [100]:
df['Displacement']=df['Displacement'].str[0].str.strip().str.replace("cc","").str.replace(",","").astype(float)

In [101]:
#df["Displacement"]=df["Displacement"].str[0].str.replace("cc","").str.replace(",","").str.strip()
#df["Displacement"]=pd.to_numeric(df["Displacement"])

In [102]:
def disp(d1,d2):
    if (d1>4000) | (d1<700) | np.isnan(d1):
        if np.isnan(d2):
            return d1
        else:
            return d2
    else:
        return d1

In [103]:
df['Displacement']=df.apply(lambda x: disp(x['Displacement'],x["sd_disp"]),axis=1)

In [104]:
df['Displacement'].isnull().sum()

180

In [105]:
df.drop(["short_description","sd_disp"],axis=1,inplace=True)

*Displacement column was cleaned and compared and matched with sd_disp (short description displacement) column. Some null values of displacement column were filled in this way*

### 4) Body Type

In [106]:
df.Body.str[1].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: Body, dtype: int64

In [107]:
df.body_type.value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [108]:
df.drop("Body",axis=1,inplace=True)

**Both the columns of "body_type" & "Body" have the same information, we will keep the "body_type" column which is more readable.**

### 5) km

In [109]:
df.km.value_counts(dropna=False)

10 km        1045
- km         1024
1 km          367
5 km          170
50 km         148
             ... 
67,469 km       1
43,197 km       1
10,027 km       1
35,882 km       1
57 km           1
Name: km, Length: 6690, dtype: int64

In [110]:
df.km=df.km.str.replace(",","").str.findall("\d+").str[0].astype(float)

*** Km column was cleaned and converted to float.**

### 6) kW

In [111]:
df.drop("kW",axis=1,inplace=True)

***Since there is no meaningful data in kW and null columns they were dropped.**

### 7) Registration- First Registration 

In [112]:
df["registration"].value_counts(dropna=False).head(5)

-/-        1597
03/2018     695
02/2019     585
05/2018     572
03/2019     543
Name: registration, dtype: int64

In [113]:
df["First Registration"].str[1].value_counts(dropna=False).head(5)

2018    4522
2016    3674
2017    3273
2019    2853
NaN     1597
Name: First Registration, dtype: int64

In [114]:
df["registration"]=df["registration"].replace("-/-",np.NaN)

In [115]:
df["age"]=pd.to_datetime(df["registration"]).dt.year

In [116]:
df.drop(["registration","First Registration"],axis=1,inplace=True)

**Year parts of registration column were extracted and these values were compared with 'First registration' column. Since all the values were exactly matched, 'First registration' column was dropped*

### 8) Prev_owner-Previous Owners

In [117]:
df["Previous Owners"]=df["Previous Owners"].str.findall("\d").str[0]

In [118]:
df['prev_owner'].str.findall("\d").str[0].value_counts(dropna=False)

1      8294
NaN    6828
2       778
3        17
4         2
Name: prev_owner, dtype: int64

In [119]:
df.drop("prev_owner",axis=1,inplace=True)

### 9) hp

In [120]:
df.hp=df.hp.str.extract('(\d{1,4})').astype('float')

### 10) Type

In [121]:
df.Type=df.Type.str[1].str.strip()

This column consists of the information about if the car is new, used, pre-registered, demonstration or employee's car. This part cleaned.
This column also includes data about fuel_type. This data was compared with fuel column. Since they exactly match, fuel datas of type column has not been taken into account.

### 11) Next Inspection - Inspection new

In [122]:
df["Next Inspection"]

0         [\n06/2021\n, \n99 g CO2/km (comb)\n]
1                                           NaN
2                                           NaN
3                                           NaN
4                                           NaN
                          ...                  
15914                                       NaN
15915    [\n01/2022\n, \n168 g CO2/km (comb)\n]
15916                                       NaN
15917                                       NaN
15918                                       NaN
Name: Next Inspection, Length: 15919, dtype: object

In [123]:
def ins(x):
    if type(x)==list:
        return x[0].strip()
    elif type(x)==float:
        return x
    else:
        return x.strip("")

In [124]:
df["Next Inspection"]=df["Next Inspection"].apply(ins)

In [125]:
def ins_n(x):
    if type(x)==list:
        return x[0].strip()
    elif type(x)==float:
        return x
    else:
        return x.strip("")

In [126]:
df['Inspection new'] = df['Inspection new'].apply(ins_n)

### 12) Warranty

In [127]:
def war(x):
    if type(x)==list:
        return x[0].str.replace("\n",np.NaN)
    elif type(x)==float:
        return x
    else:
        return x

In [128]:
df["Warranty"]=df.Warranty.apply(lambda x : x[0] if type(x)==list else x)

In [129]:
df["Warranty"]=df.Warranty.str.strip("\n").str.extract('(\d{1,2})').astype("float")

### 13) Full Service, Non-smoking Vehicle, Offer Number

In [130]:
df.drop('Full Service',axis=1,inplace=True)

In [131]:
df.drop('Non-smoking Vehicle',axis=1,inplace=True)

In [132]:
df.drop('Offer Number',axis=1,inplace=True)

* Since there is no meaningful data in these columns, they were dropped.

### 13) Body Color -Body Color Original

In [133]:
df["Body Color"]=df["Body Color"].str[1]

In [134]:
df["Body Color Original"]=df["Body Color Original"].str[0]

In [135]:
df["Body Color Original"]=df["Body Color Original"].str.strip()

In [136]:
df.drop(["Body Color","Body Color Original"], axis=1, inplace=True)

### 14) Nr. of Doors -Nr. of Seats -Displacement - Gearing Type-Model Code-Drive chain-Paint Type-Emission Class-Cylinders

In [137]:
df["Paint Type"]=df["Paint Type"].str[0].str.strip()

In [138]:
df["Nr. of Seats"]=df["Nr. of Seats"].str[0].str.strip()
df["Nr. of Seats"]=pd.to_numeric(df["Nr. of Seats"])

In [139]:
df["Gearing Type"]=df["Gearing Type"].str[1].str.strip()

In [140]:
df["Nr. of Doors"]=df["Nr. of Doors"].str[0].str.strip()
df["Nr. of Doors"]=pd.to_numeric(df["Nr. of Doors"])

In [141]:
df["Model Code"]=df["Model Code"].str[0].str.strip()
df.drop("Model Code",axis=1,inplace=True)

In [142]:
df["Drive chain"]=df["Drive chain"].str[0].str.strip()

In [143]:
df["Cylinders"]=df["Cylinders"].str[0].str.strip()

In [144]:
df["Cylinders"]=pd.to_numeric(df["Cylinders"])

In [145]:
df['Weight'] = df['Weight'].str[0].str.replace(',','').str.findall('\d+').str[0].astype('float')

### 15) Upholstery

In [146]:
df["Upholstery"] = [item[0] if type(item) == list else item for item in df.Upholstery]

In [147]:
df["Upholstery"] = df.Upholstery.str.strip("\n").str.split(", ")

In [148]:
u_type = ["Cloth", 'Part leather', 'Full leather', 'Velour', 'alcantara']

df["Upholstery_type"] = df["Upholstery"].apply(lambda x : x[0] if type(x) == list and x[0] in u_type else np.nan)

In [149]:
color = ['Black', 'Grey', 'Brown', 'Beige', 'White', 'Blue', 'Red', 'Yellow', 'Orange']

def color_finder(x):
    if type(x) == list and len(x) == 2:
        return x[1]
    elif type(x) == list and x[0] in color:
        return x[0]
    else:
        return np.nan
    
df['Upholstery_color'] = df.Upholstery.apply(color_finder)

In [150]:
df.drop("Upholstery", axis=1, inplace=True)

### 16) Fuel

In [151]:
df["Fuel"]=df["Fuel"].str[1]

In [152]:
def func2(x):
    if 'Diesel' in x:
        return 'Diesel'
    elif 'Super' in x:
        return 'Benzine'
    elif 'Gasoline' in x:
        return 'Benzine'
    elif 'Benzine' in x:
        return 'Benzine'
    else:
        return x

In [153]:
df["Fuel"]=df["Fuel"].apply(func2)

In [154]:
df['Fuel'].replace(['CNG','LPG', 'Liquid petroleum gas (LPG)','CNG (Particulate Filter)', 'Domestic gas H','Biogas'], 'Gas',inplace = True)

In [155]:
df['Fuel'].replace(['Others (Particulate Filter)','Others','Electric'], 'Others',inplace = True)

### 17) Consumption

In [156]:
def consume_combined(a):
    if type(a) == list:
        if len(a) >3:
            for i in a:
                if 'comb' in i:
                    return i
        else:
            return a[0]            
    
    else:
        return a
    
df['Consumption_combined'] = df['Consumption'].apply(consume_combined)

In [157]:
def cleaning_consumption(a):
    if type(a) == list:
        if len(a) > 0:
            b = re.findall("\d\.?\d?", a[0])
            return b[0]
        else:
            return np.nan
    elif type(a) == str:
        b = re.findall("\d\.?\d?",a)
        return b[0]        
    else:
        return a

In [158]:
def consume_city(a):
    if type(a) == list:
        if len(a) >3:
            for i in a:
                if 'city' in i:
                    return i
        else:
            return a[1]           
    
    else:
        return a
    
df['Consumption_city'] = df['Consumption'].apply(consume_city)

In [159]:
def consume_country(a):
    if type(a)== list:
        if len(a) >3:
            for i in a:
                if 'country' in i:
                    return i
        else:
            return a[2]            
    
    else:
        return a
    
df['Consumption_country'] = df['Consumption'].apply(consume_country)

In [160]:
df['Consumption_combined'] = df['Consumption_combined'].apply(cleaning_consumption).astype('float')
df['Consumption_city'] = df['Consumption_city'].apply(cleaning_consumption).astype('float')
df['Consumption_country'] = df['Consumption_country'].apply(cleaning_consumption).astype('float')

df.drop("Consumption",axis=1,inplace=True)

### 18 ) CO2 Emission

In [161]:
df['CO2 Emission'] = df['CO2 Emission'].str[0].str.findall('\d+').str[0].astype('float')

### 19) Emission Class

In [162]:
df["Emission Class"]=df["Emission Class"].str[0].str.replace("\n","")

In [163]:
df['Emission Class'].replace(['Euro 6','Euro 6d-TEMP','Euro 6d', 'Euro 6c'], 'Euro 6', inplace = True)

### 20) Emission Label - Description - Last Timing Belt Service Date- Electricity consumption- Available from- Availability- Other Fuel Types-Last Service Date

In [164]:
df.drop("Emission Label",axis=1,inplace=True)
df.drop("description",axis=1,inplace=True)
df.drop("Last Timing Belt Service Date",axis=1,inplace=True)
df.drop("Electricity consumption",axis=1,inplace=True)
df.drop("Available from",axis=1,inplace=True)
df.drop("Availability",axis=1,inplace=True)
df.drop("Other Fuel Types",axis=1,inplace=True)
df.drop("Last Service Date",axis=1,inplace=True)

### 21) Gears

In [165]:
df["Gears"]=df["Gears"].str[0].str.replace("\n","")

In [166]:
df["Gears"]=pd.to_numeric(df["Gears"])

In [167]:
df.loc[14495,"Gears"]=5.0

In [168]:
df.drop("Country version", axis=1, inplace=True)

### 22) Comfort_Convenience-Entertainment_Media-Extras-Safety Security

In [169]:
df["Comfort & Convenience"] = [",".join(item) if type(item) == list else item for item in df["Comfort & Convenience"]]

In [170]:
df["Entertainment & Media"] = [",".join(item) if type(item) == list else item for item in df["Entertainment & Media"]]

In [171]:
df["Extras"] = [",".join(item) if type(item) == list else item for item in df["Extras"]]

In [172]:
df["Safety & Security"] = [",".join(item) if type(item) == list else item for item in df["Safety & Security"]]

In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   make_model             15919 non-null  object 
 1   body_type              15859 non-null  object 
 2   price                  15919 non-null  int64  
 3   vat                    11406 non-null  object 
 4   km                     14895 non-null  float64
 5   hp                     15831 non-null  float64
 6   Type                   15917 non-null  object 
 7   Previous Owners        9049 non-null   object 
 8   Next Inspection        3535 non-null   object 
 9   Inspection new         3932 non-null   object 
 10  Warranty               4853 non-null   float64
 11  Paint Type             10147 non-null  object 
 12  Nr. of Doors           15707 non-null  float64
 13  Nr. of Seats           14942 non-null  float64
 14  Gearing Type           15919 non-null  object 
 15  Di

In [174]:
df.to_csv("Capstone_new.csv",index=False)

In [176]:
for i in df.columns:
    print(i)

make_model
body_type
price
vat
km
hp
Type
Previous Owners
Next Inspection
Inspection new
Warranty
Paint Type
Nr. of Doors
Nr. of Seats
Gearing Type
Displacement
Cylinders
Weight
Drive chain
Fuel
CO2 Emission
Emission Class
Comfort & Convenience
Entertainment & Media
Extras
Safety & Security
Gears
age
Upholstery_type
Upholstery_color
Consumption_combined
Consumption_city
Consumption_country


In [177]:
len(df.columns)

33